<a href="https://colab.research.google.com/github/ThanhHung2112/Indeed_crawler/blob/main/Skills_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -m spacy download en_core_web_lg

2024-01-16 15:51:34.079312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 15:51:34.079380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 15:51:34.081346: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 15:51:35.487562: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import spacy
from spacy.training.example import Example
import json

In [ ]:
# Load a pre-trained spaCy model
nlp = spacy.blank("en")

# Add the NER component to the pipeline
ner = nlp.add_pipe("ner")

# Load training data from JSON file
def load_data_from_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file_path}: {e}")
        return None

# Try loading the data
training_data = load_data_from_json("/content/job_skill_ner.json")

# Check if data is loaded successfully
if training_data is not None:
    print(f"Loaded {len(training_data)} examples from the JSON file.")
else:
    print("Failed to load data. Please check the JSON file and try again.")

# Rest of your code (combining data, converting to spaCy format, training the model, etc.)


# # Convert combined data to spaCy format
combined_examples = []
for entry in combined_data:
    text = entry["text"]
    entities = entry["entities"]
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, {"entities": entities})
    combined_examples.append(example)

# Train the NER model with the combined data
nlp.begin_training()
for _ in range(10):  # Adjust the number of epochs
    for example in combined_examples:
        nlp.update([example], drop=0.5)  # Adjust dropout

# Save the trained model
nlp.to_disk("job_skills_ner_model")


In [3]:
import spacy
from spacy.training.example import Example
import json
import random
import warnings

# Suppress warnings (optional, use with caution)
warnings.filterwarnings("ignore", category=UserWarning, module="spacy")
warnings.filterwarnings("ignore", category=FutureWarning, module="tensorflow")

# Load training data from JSON file
def load_data_from_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file_path}: {e}")
        return None

# Load the spaCy English model
nlp = spacy.blank("en")

# Add the NER component to the pipeline
ner = nlp.add_pipe("ner")

# Add the labels (in this case, just "SKILL")
ner.add_label("SKILL")

# Disable other pipeline components for training efficiency
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Specify the path to your JSON file
json_file_path = "/content/skill.json"

# Load training data from JSON file
training_data = load_data_from_json(json_file_path)

if training_data:
    # Remove duplicate entities based on start and end positions
    cleaned_training_data = []
    seen_entities = set()

    for entry in training_data:
        labels_list = entry.get("label", [])

        # Ensure that labels is a list of dictionaries
        if isinstance(labels_list, str):
            labels_list = [{"text": labels_list, "start": -1, "end": -1, "labels": ["SKILL"]}]
        elif not isinstance(labels_list, list):
            continue  # Skip entries without a valid label list

        cleaned_labels_list = []
        for label in labels_list:
            start = label.get("start", -1)
            end = label.get("end", -1)
            label_text = label.get("labels", "")

            # Ensure unique start and end positions
            if (start, end) not in seen_entities:
                cleaned_labels_list.append({"text": label_text, "start": start, "end": end, "labels": ["SKILL"]})
                seen_entities.add((start, end))

        if cleaned_labels_list:
            cleaned_training_data.append({"text": entry.get("text", ""), "label": cleaned_labels_list})

    with nlp.disable_pipes(*other_pipes):
        # Initialize the training
        nlp.begin_training()

        # Iterate through the cleaned training data for multiple epochs
        for epoch in range(25):
            # Shuffle the cleaned training data
            random.shuffle(cleaned_training_data)

            # Create batches of cleaned training data
            for entry in cleaned_training_data:
                text = entry.get("text", "")
                labels_list = entry.get("label", [])

                # Process annotations and extract entities
                entities = []
                for label in labels_list:
                    start = label.get("start", -1)
                    end = label.get("end", -1)
                    label_text = label.get("labels", "")
                    entities.append((start, end, label_text))

                # Create a spaCy Example
                example = Example.from_dict(nlp.make_doc(text), {"entities": entities})

                # Update the model with iterating each example
                nlp.update([example], drop=0.5)

    # Save the trained model to disk /content/drive/MyDrive/NER/
    nlp.to_disk("/content/drive/MyDrive/NER/skills_ner_model·")
else:
    print("Failed to load training data from the JSON file.")


In [4]:
""
# Load the saved model from disk
loaded_nlp = spacy.load("/content/drive/MyDrive/NER/skills_ner_model")

# Test the loaded model with some example texts
test_texts = [
    "I am skilled in Python and Java programming.",
    "My experience includes using TensorFlow for machine learning.",
    "I have hands-on experience with MongoDB and MySQL.",
    "Backend development with Node.js and Django is my expertise.",
    "Build machine learning",
    """
    As Technical Account Manager you will support our internal and external clients in online marketing campaigns configuration. We are working on our own proprietary solution that is the main point of client’s Marketing Technology Stack, integrating data from various services and delivering this data to external providers.

Tasks

Analyzing client websites to make sure they have the data needed for online marketing campaigns.
Setting up, running, checking, and turning off various services for clients.
Solving issues with these services.
Working with our developer teams to handle reported problems, configure services, improve existing solutions, and brainstorm new ones.
Checking problem reports and passing them to the right teams with initial analysis results.
Looking at services from external vendors that we haven't used yet when we need them.
Completing tasks with clear documentation for reference in the future.
Opportunity to become the go-to person for a particular area of our knowledge base and manage internal team documentation.
Actively participating in team meetings where we all discuss team matters (Governance Meeting).
Requirements

You should know about internet browsers (understand what happens when you enter a website URL and hit "Enter").
You should be comfortable using Developer tools in browsers (able to analyze network traffic and inspect data layer - like JavaScript variables and DOM elements variables).
You should be familiar with Online Marketing tools, such as:
Principles of Tag management systems (like Google Tag Manager, which is a must, and others like Tealium or Commanders Act)
Analytics tools like Google Analytics
Online Marketing platforms, like Google Ads, Facebook Advertising, and others (Criteo, Google Marketing Platform, Microsoft Advertising, etc.)
Consent Management Platforms (CMP) like Usercentrics, Klaro, OneTrust, Didomi
You should have some knowledge of SQL (to write basic queries for database data retrieval).
You should have some understanding of JavaScript (or a willingness to learn) to handle monitoring code implementation (tracking data, processing, using variables from DOM or URL).
You should be knowledgeable about online privacy topics, especially regarding tracking codes and Privacy Enhanced Technologies in various browsers.
Fluency in English is a must.
You should be able to document your work so that you or anyone else can pick up where you left off and retain valuable information for future reference (knowledge capture).
Understanding processes related to IT service management (like ITIL) is a plus.
You should be a good team player, adhering to mutually agreed-upon rules and be able to communicate on technical topics with both technical and non-technical individuals.
Knowledge of the German language is an added advantage.
You'll help maintain the team and company's positive image.
You should understand and support the team's vision, to which you will be contributing.
Benefits

Work in a global team.
remote work first company.
Competitive pay.
Enjoy perks like a MultiSport card and private health coverage.
Grow professionally with learning opportunities.
Join a friendly, energetic team with a laid-back work environment
    """,
    "Client should have strong problem solving skills, and good at both reading and speaking English"
]

for text in test_texts:
    doc = loaded_nlp(text)
    print("Input Text:", text)
    print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])
    print()


Input Text: I am skilled in Python and Java programming.
Entities: [('Python', "['SKILL']"), ('Java programming', "['SKILL']")]

Input Text: My experience includes using TensorFlow for machine learning.
Entities: [('machine learning.', "['SKILL']")]

Input Text: I have hands-on experience with MongoDB and MySQL.
Entities: [('MySQL', "['SKILL']")]

Input Text: Backend development with Node.js and Django is my expertise.
Entities: []

Input Text: Build machine learning
Entities: [('machine learning', "['SKILL']")]

Input Text: 
    As Technical Account Manager you will support our internal and external clients in online marketing campaigns configuration. We are working on our own proprietary solution that is the main point of client’s Marketing Technology Stack, integrating data from various services and delivering this data to external providers.

Tasks

Analyzing client websites to make sure they have the data needed for online marketing campaigns.
Setting up, running, checking, and tu